In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers = tickers.to_list()

In [ ]:
df = yf.download(tickers,start= '2010-01-01', end ='2023-04-06')


PIvot Table in Pandas (How to slice)

In [ ]:
df

In [ ]:
# This is a proof of concept of df slicing before abstracting it in a function 
#df fiiltered by( the columns filterd by (were does the lvl 1 column header value say APPLE))
df[df.columns[df.columns.get_level_values(1) == 'AAPL']]

In [ ]:
def slice_by_ticker(ticker,df):
    unpivot = df.copy()
    unpivot = unpivot[unpivot.columns[unpivot.columns.get_level_values(1) == ticker]]
    unpivot.columns = unpivot.columns.droplevel(1)
    unpivot['Price'] = unpivot['Open'].shift(-1)
    return unpivot

In [ ]:
AAPL = slice_by_ticker('AAPL',df)
AAPL

In [ ]:
def MA_calc(df,n,m):
    df[f'MA{n}'] = df['Adj Close'].rolling(n).mean()
    df[f'MA{m}'] = df['Adj Close'].rolling(m).mean()
    df.dropna(inplace= True)
    return df

In [ ]:
AAPL = MA_calc(AAPL,50,200)
AAPL

buyprice, sellprice = [],[]
buydate, selldate = [], []

in_positon = False

for index, rows in AAPL.iterrows():
    if not in_positon and (rows[-2] > rows[-1]):
        buyprice.append(rows['Price'])
        buydate.append(rows.name)
        in_positon = True
    if in_positon and (rows[-2] > rows[-1]):
        sellprice.append(rows['Price'])
        selldate.append(rows.name)
        in_positon = False

In [ ]:
def backtest(df, n, m):
    MA_calc(df,n,m)
    in_position = False
    profits = []
    
    for index,row in df.iterrows():
        if not in_position:
            if (row[-2] > row[-1]):
                buyprice = row['Price']
                in_position = True
        if in_position:
            if (row[-2] < row[-1]):
                profit = (row['Price'] / buyprice)/-1
                profits.append(profit)
                in_position = False

    gain = (pd.Series(profits) + 1).prod()
    return gain

In [ ]:
ret = backtest(slice_by_ticker('AAPL',df),50,100)

In [ ]:
ret

In [ ]:
row = AAPL.iloc[-1]
row

In [ ]:
row[-2] > row[-1]